In [2]:
from jinja2 import (
    Environment,
    select_autoescape,
    FileSystemLoader,
    meta
)

In [3]:
env_template = Environment(
    loader=FileSystemLoader('jinja_prompt'),
    autoescape=select_autoescape
)

In [4]:
def get_required_vars(environ, template_file):
    temp_source = environ.loader.get_source(environ,
                                            template_file)
    parsed_content = environ.parse(temp_source)
    return meta.find_undeclared_variables(parsed_content)

In [5]:
def load_render_template(environ,
                         template_file,
                         context):
    template = environ.get_template(template_file)
    return template.render(**context)

In [6]:
import json
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv

In [7]:
model_used_eco = "gpt-3.5-turbo"
model_used_super = "gpt-4-1106-preview"
programming_expert = """You are expert in coding a given task using the information provided to you"""

In [8]:
def llm_call_openai(user_message: str,
                    system_message: str,
                    model_used: str):
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    try:
        response = client.chat.completions.create(
            model=model_used,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "system", "content": user_message}
            ],
            temperature=0.0,
            top_p=1,
            frequency_penalty=0.1,
            presence_penalty=0.1,
        )
        assistant_message = response.choices[0].message.content
        print(f"Tokens used: {response.usage.total_tokens}")
        global total_tokens
        total_tokens += response.usage.total_tokens
        return {"response": assistant_message}
    except Exception as e:
        print(e)
        return {"error": "An error occurred while processing the request."}

In [9]:
env_template.list_templates()

['code_extraction.prompt',
 'data_analysis.prompt',
 'feature_plan.prompt',
 'specs.prompt',
 'task_classifying.prompt']

In [10]:
from tavily import TavilyClient
load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")

tavily = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])

In [11]:
question = "Split a text file with list of tasks that are seperated with \\n using python code"

In [12]:
response = tavily.search(query=question)

for data in response['results']:
    print(data['title'])
    print(data['url'])
    print(data['content'])

Python 3 split('\n') - Stack Overflow
https://stackoverflow.com/questions/39051835/python-3-split-n
Here's a list of the file contents:\n{}".format(file_list)) Any help would be greatly appreciated, thank you. If it helps to explain what I am dealing with, here's the contents of the stats.csv file:
The Fastest Way to Split a Text File Using Python
https://www.pythonforbeginners.com/files/the-fastest-way-to-split-a-text-file-using-python
PythonForBeginners.com
Learn By Example
The Fastest Way to Split a Text File Using Python
Author: Josh Petty Last Updated: June 5, 2023
Python is one of the most popular programming languages in the world. In order to perform the slice, we will also use the len() method to find the total number of lines in the original file.
 Once we get the data via list comprehension, we will use the split() method to split the lines and add them to a new list as shown below.
 Using a Generator to Split a Text File in Python
In Python, a generator is a special routine

In [13]:
search_out = []
for cont in response['results']:
    # print(cont['snippet'])
    search_out.append((cont['content']))

In [15]:
print(search_out[0])

Here's a list of the file contents:\n{}".format(file_list)) Any help would be greatly appreciated, thank you. If it helps to explain what I am dealing with, here's the contents of the stats.csv file:


In [16]:
coder_temp = env_template.get_template("code_extraction.prompt")

In [ ]:
print(coder_temp.render())

In [18]:
context = {"net_data":search_out,
           "question":question}

rendered_prompt = coder_temp.render(**context)
print(rendered_prompt)

You are expert coder and researcher. You are approached by the program manager with the below question and requires you to write code
Split a text file with list of tasks that are seperated with \n using python code

You have searched about the question and found the below information. Using that write the code as requested by program manager 

Here&#39;s a list of the file contents:\n{}&#34;.format(file_list)) Any help would be greatly appreciated, thank you. If it helps to explain what I am dealing with, here&#39;s the contents of the stats.csv file:

PythonForBeginners.com
Learn By Example
The Fastest Way to Split a Text File Using Python
Author: Josh Petty Last Updated: June 5, 2023
Python is one of the most popular programming languages in the world. In order to perform the slice, we will also use the len() method to find the total number of lines in the original file.
 Once we get the data via list comprehension, we will use the split() method to split the lines and add them to a

In [19]:
total_tokens = 0

get_code = llm_call_openai(programming_expert,
                           rendered_prompt,
                           model_used_eco)

Tokens used: 806


In [20]:
responses = get_code['response']

In [21]:
print(responses)

```python
# Python code to split a text file into a list of tasks

# Read the contents of the file
with open('file.txt', 'r') as file:
    file_contents = file.read()

# Split the file contents into a list of tasks
tasks_list = file_contents.split('\n')

# Print the list of tasks
print(tasks_list)
```


In [22]:
code_part = responses[9:-3]
print(code_part)


# Python code to split a text file into a list of tasks

# Read the contents of the file
with open('file.txt', 'r') as file:
    file_contents = file.read()

# Split the file contents into a list of tasks
tasks_list = file_contents.split('\n')

# Print the list of tasks
print(tasks_list)



In [32]:
with open('python_code.py', 'w',) as wds:
    wds.write(code_part)